In [1]:
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

## Aplicación

In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [56]:
# Create a dash application
app = dash.Dash(__name__)

In [57]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                             options=[{'label': 'All Sites', 'value': 'ALL'},
                                                      {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                      {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                      {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                      {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
                                             value='ALL',
                                             placeholder='Select a Launch Site here',
                                             searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                            
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                #TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                     10000: '10000'},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart'))
                                ])

In [58]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    
    if entered_site == 'ALL':
        all_spacex_df = spacex_df[spacex_df['class'] == 1]
        fig = px.pie(all_spacex_df, values='class', 
                     names='Launch Site', 
                     title='Succesful launches for each site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_spacex_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        class_df = filtered_spacex_df['class'].value_counts().reset_index()
        fig = px.pie(class_df, values='count', 
                     names = 'class', 
                     title = 'Successful launches vs Unsuccessful launches')
        return fig

In [59]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
               Input(component_id="payload-slider", component_property="value")])
def get_scatter(entered_site, payload):
    
    if entered_site == 'ALL':
        all_spacex_df = spacex_df[(payload[0] <= spacex_df['Payload Mass (kg)']) 
                                  & (spacex_df['Payload Mass (kg)'] <= payload[1])]
        fig = px.scatter(all_spacex_df, 
                         x='Payload Mass (kg)',
                         y='class', 
                         color="Booster Version Category")
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_spacex_df = spacex_df[(spacex_df['Launch Site'] == entered_site) & 
                                       (payload[0] <= spacex_df['Payload Mass (kg)']) & 
                                       (spacex_df['Payload Mass (kg)'] <= payload[1])]
        fig = px.scatter(filtered_spacex_df, 
                         x='Payload Mass (kg)',
                         y='class', 
                         color="Booster Version Category")
        return fig

In [60]:
# Run the app
if __name__ == '__main__':
    app.run(jupyter_mode="tab")

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>

## Pruebas

In [48]:
t_fltr = spacex_df[(spacex_df['Launch Site'] == 'CCAFS SLC-40') & (1000 <= spacex_df['Payload Mass (kg)']) 
                    & (spacex_df['Payload Mass (kg)'] <= 6000)]
t_fltr.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
49,49,45,CCAFS SLC-40,1,2205.00,F9 FT B1035.2,FT
50,50,47,CCAFS SLC-40,1,3696.65,F9 B4 B1043.1,B4
51,51,48,CCAFS SLC-40,0,4230.00,F9 FT B1032.2,FT
53,53,52,CCAFS SLC-40,0,2647.00,F9 B4 B1039.2,B4
55,55,56,CCAFS SLC-40,0,5384.00,F9 B4 B1040.2,B4


In [42]:
t_fltr

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
49,49,45,CCAFS SLC-40,1,2205.00,F9 FT B1035.2,FT
50,50,47,CCAFS SLC-40,1,3696.65,F9 B4 B1043.1,B4
51,51,48,CCAFS SLC-40,0,4230.00,F9 FT B1032.2,FT
52,52,50,CCAFS SLC-40,0,6092.00,F9 B4 B1044,B4
53,53,52,CCAFS SLC-40,0,2647.00,F9 B4 B1039.2,B4
54,54,53,CCAFS SLC-40,1,362.00,F9 B4 B1045.1,B4
55,55,56,CCAFS SLC-40,0,5384.00,F9 B4 B1040.2,B4
